In [8]:
import torch
from torch_geometric.loader import DataLoader
import os
import yaml
import pandas as pd
from gridFM.datasets.powergrid import GridDataset
from gridFM.io.param_handler import parse_yaml, load_normalizer
from gridFM.datasets.globals import *
import torch.nn.functional as F

In [20]:
# Replace <run_name> with the correct folder name for your run
run_name = "2024-11-18_17-47-39"

run_dir = "../runs/" + run_name
config_path = os.path.join(run_dir, "config/config.yaml")
test_indices_path = os.path.join(run_dir, "data_idx/test_indices.csv")
model_path = os.path.join(run_dir, "model.pth")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
args = parse_yaml(config_path)

In [22]:
node_normalizer, edge_normalizer = load_normalizer(args=args)

In [23]:
data_path = os.path.join(os.getcwd(), "..", "data", args.network)

dataset = GridDataset(
        root=data_path,
        scenarios=args.data.scenarios,
        norm_method=args.data.normalization,
        node_normalizer=node_normalizer,
        edge_normalizer=edge_normalizer,
        mask_ratio=args.data.mask_ratio,
        mask_dim=args.data.mask_dim,
        mask_value=args.data.mask_value
    )


In [24]:
test_indices = pd.read_csv(test_indices_path)["index"].tolist()
test_dataset = torch.utils.data.Subset(dataset, test_indices)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [25]:
model = torch.load(model_path).to(device)
model.eval()


C:\Users\MatteoMazzonelli\AppData\Local\Temp\ipykernel_35424\908644874.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path).to(device)


GraphTransformer(
  (layers): ModuleList(
    (0): TransformerConv(9, 32, heads=8)
    (1-2): 2 x TransformerConv(256, 32, heads=8)
  )
  (mlps): Sequential(
    (0): Linear(in_features=256, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [26]:
# Initialize lists to store losses for each node type
RMSE_loss_PQ = []
RMSE_loss_PV = []
RMSE_loss_REF = []
MAE_loss_PQ = []
MAE_loss_PV = []
MAE_loss_REF = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        
        # Prepare input features with the same masking logic as in training
        input_features = torch.cat((batch.y, batch.x[:, 6:]), dim=1)
        
        # Masking logic with random values
        mask_PQ = input_features[:, PQ] == 1
        mask_PV = input_features[:, PV] == 1
        mask_REF = input_features[:, REF] == 1

        input_features[mask_PQ, VM] = args.data.mask_value
        input_features[mask_PQ, VA] = args.data.mask_value

        input_features[mask_PV, QG] = args.data.mask_value
        input_features[mask_PV, VA] = args.data.mask_value

        input_features[mask_REF, PG] = args.data.mask_value
        input_features[mask_REF, QG] = args.data.mask_value


        # Forward pass
        output = model(input_features, batch.edge_index, batch.edge_attr)
        
        # Denormalize the output and target
        output_denorm = node_normalizer.inverse_transform(output.cpu())
        target_denorm = node_normalizer.inverse_transform(batch.y.cpu())
        
        # Compute per-feature RMSE and MAE for each node type
        if mask_PQ.any():  # Check if any nodes of type PQ exist in this batch
            RMSE_loss_PQ.append(F.mse_loss(output_denorm[mask_PQ.cpu()], target_denorm[mask_PQ.cpu()], reduction='none'))
            MAE_loss_PQ.append(torch.abs(output_denorm[mask_PQ.cpu()] - target_denorm[mask_PQ.cpu()]))

        if mask_PV.any():  # Check if any nodes of type PV exist in this batch
            RMSE_loss_PV.append(F.mse_loss(output_denorm[mask_PV.cpu()], target_denorm[mask_PV.cpu()], reduction='none'))
            MAE_loss_PV.append(torch.abs(output_denorm[mask_PV.cpu()] - target_denorm[mask_PV.cpu()]))

        if mask_REF.any():  # Check if any nodes of type REF exist in this batch
            RMSE_loss_REF.append(F.mse_loss(output_denorm[mask_REF.cpu()], target_denorm[mask_REF.cpu()], reduction='none'))
            MAE_loss_REF.append(torch.abs(output_denorm[mask_REF.cpu()] - target_denorm[mask_REF.cpu()]))

# Calculate mean losses for each node type across batches
def compute_mean_loss(loss_list):
    return torch.sqrt(torch.cat(loss_list, dim=0).mean(dim=0)) if loss_list else None

RMSE_pq = compute_mean_loss(RMSE_loss_PQ).tolist()
RMSE_pv = compute_mean_loss(RMSE_loss_PV).tolist()
RMSE_ref = compute_mean_loss(RMSE_loss_REF).tolist()

MAE_pq = torch.cat(MAE_loss_PQ, dim=0).mean(dim=0).tolist() if MAE_loss_PQ else None
MAE_pv = torch.cat(MAE_loss_PV, dim=0).mean(dim=0).tolist() if MAE_loss_PV else None
MAE_ref = torch.cat(MAE_loss_REF, dim=0).mean(dim=0).tolist() if MAE_loss_REF else None

overall_RMSE_loss = RMSE_loss_PQ + RMSE_loss_PV + RMSE_loss_REF
overall_RMSE = compute_mean_loss(overall_RMSE_loss).tolist()

# Calculate overall MAE by combining all individual losses across node types
overall_MAE_loss = MAE_loss_PQ + MAE_loss_PV + MAE_loss_REF
overall_MAE = torch.cat(overall_MAE_loss, dim=0).mean(dim=0).tolist() if overall_MAE_loss else None


In [27]:
# Define feature labels for readability
feature_labels = ["Pd", "Qd", "Pg", "Qg", "Vm", "Va"]

# Prepare the data in the desired format
data = {
    'Metric': [
        'RMSE - PQ', 'RMSE - PV', 'RMSE - REF',
        'MAE - PQ', 'MAE - PV', 'MAE - REF',
        'Overall RMSE', 'Overall MAE',
    ],
    'Pd (MW)': [
        RMSE_pq[0], RMSE_pv[0], RMSE_ref[0],
        MAE_pq[0], MAE_pv[0], MAE_ref[0],
        overall_RMSE[0], overall_MAE[0],
    ],
    'Qd (MVar)': [
        RMSE_pq[1], RMSE_pv[1], RMSE_ref[1],
        MAE_pq[1], MAE_pv[1], MAE_ref[1],
        overall_RMSE[1], overall_MAE[1],
    ],
    'Pg (MW)': [
        RMSE_pq[2], RMSE_pv[2], RMSE_ref[2],
        MAE_pq[2], MAE_pv[2], MAE_ref[2],
        overall_RMSE[2], overall_MAE[2],
    ],
    'Qg (MVar)': [
        RMSE_pq[3], RMSE_pv[3], RMSE_ref[3],
        MAE_pq[3], MAE_pv[3], MAE_ref[3],
        overall_RMSE[3], overall_MAE[3],
    ],
    'Vm (p.u.)': [
        RMSE_pq[4], RMSE_pv[4], RMSE_ref[4],
        MAE_pq[4], MAE_pv[4], MAE_ref[4],
        overall_RMSE[4], overall_MAE[4],
    ],
    'Va (degree)': [
        RMSE_pq[5], RMSE_pv[5], RMSE_ref[5],
        MAE_pq[5], MAE_pv[5], MAE_ref[5],
        overall_RMSE[5], overall_MAE[5],
    ]
}


# Create a DataFrame where rows are loss types and columns are features
df_results = pd.DataFrame(data)

In [28]:
df_results

,Metric,Pd (MW),Qd (MVar),Pg (MW),Qg (MVar),Vm (p.u.),Va (degree)
0,RMSE - PQ,360.672729,117.283630,554.618164,268.383209,0.016351,2.571947
1,RMSE - PV,883.103271,328.457794,1300.323730,308.921783,0.013570,5.805666
2,RMSE - REF,355.012207,39.442406,465.651428,659.721924,0.025224,1.775424
3,MAE - PQ,311.581909,90.175583,511.588959,211.353073,0.014059,1.931597
4,MAE - PV,652.729370,239.734329,838.834473,256.809326,0.010429,3.206569
5,MAE - REF,349.068420,38.612995,462.267395,659.666077,0.025222,1.764609
6,Overall RMSE,803.301392,296.629181,1183.859863,321.115112,0.014632,5.289740
7,Overall MAE,583.439636,207.013153,769.427734,262.863647,0.011565,2.937023


In [ ]:
df_results

,Metric,Pd (MW),Qd (MVar),Pg (MW),Qg (MVar),Vm (p.u.),Va (degree)
0,RMSE - PQ,25.368912,5.111722,26.893908,5.785166,0.000452,0.217610
1,RMSE - PV,191.729324,7.013256,201.273987,15.746964,0.007344,0.469672
2,RMSE - REF,12.245392,4.295181,11.550007,4.631992,0.000141,0.175780
3,MAE - PQ,18.604321,4.000228,21.221478,4.778119,0.000376,0.155650
4,MAE - PV,82.033867,5.587916,95.118225,10.363422,0.003667,0.245801
5,MAE - REF,8.210846,4.272607,11.392878,4.623018,0.000136,0.175550
6,Overall RMSE,171.086899,6.644597,179.607620,14.253899,0.006543,0.429165
7,Overall MAE,68.552116,5.268822,79.490334,9.202497,0.002977,0.227835
8,max_value,13599.701172,2902.000000,12245.000000,2441.497803,1.100000,52.132515
9,min_value,62.186268,37.400002,0.000000,-612.789978,0.991608,-5.356375


In [ ]:
df_results

,Metric,Pd (MW),Qd (MVar),Pg (MW),Qg (MVar),Vm (p.u.),Va (degree)
0,RMSE - PQ,25.368914,5.111723,26.893908,5.785166,0.000452,0.217610
1,RMSE - PV,191.729324,7.013257,201.273987,15.746964,0.007344,0.469672
2,RMSE - REF,12.245390,4.295181,11.550012,4.631990,0.000141,0.175780
3,MAE - PQ,18.604321,4.000229,21.221478,4.778119,0.000376,0.155650
4,MAE - PV,82.033867,5.587917,95.118217,10.363423,0.003667,0.245801
5,MAE - REF,8.210844,4.272606,11.392881,4.623015,0.000136,0.175551
6,Overall RMSE,171.086899,6.644598,179.607620,14.253899,0.006543,0.429165
7,Overall MAE,68.552124,5.268823,79.490334,9.202494,0.002977,0.227835
8,max_value,13599.701172,2902.000000,12245.000000,2441.497803,1.100000,52.132515
9,min_value,62.186268,37.400002,0.000000,-612.789978,0.991608,-5.356375


In [ ]:
df_results

,Metric,Pd (MW),Qd (MVar),Pg (MW),Qg (MVar),Vm (p.u.),Va (degree)
0,RMSE - PQ,98.120064,14.808173,127.881615,31.273409,0.001199,0.419745
1,RMSE - PV,214.892426,22.535358,224.427902,37.581810,0.001372,1.916810
2,RMSE - REF,44.746571,6.032668,49.520382,9.918824,0.000065,0.078175
3,MAE - PQ,73.605324,12.281806,105.695038,21.189455,0.000997,0.316331
4,MAE - PV,130.090820,15.866342,155.156601,28.468706,0.001073,0.726655
5,MAE - REF,31.486374,3.944438,47.777870,9.195753,0.000042,0.066452
6,Overall RMSE,195.840500,21.019842,207.005112,35.947056,0.001320,1.715977
7,Overall MAE,116.951797,14.837219,142.926041,26.549080,0.001024,0.633144
8,max_value,13599.701172,2902.000000,12245.000000,2441.497803,1.100000,52.132515
9,min_value,62.186268,37.400002,0.000000,-612.789978,0.991608,-5.356375
